In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-marathi/classification


In [7]:
path = Path('./')

In [8]:
df_train = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/mr/mr-train.csv', header=None)
df_train.head()

,0,1
0,entertainment,…म्हणून सानिया मिर्झाची ड्यू डेट आहे खास
1,entertainment,Video : दीपिका-रणवीरच्या लग्नाचा मेन्यू झाला लीक!
2,sports,"सचिनच्या रणजी कारकीर्दीचा शेवट गोड, मुंबईचा विजय"
3,state,पुरंदरेंना महाराष्ट्र भूषण पुरस्काराविरोधात सं...
4,sports,"जयपूर पोलिसांच्या जाहिरातीवर बुमराह संतापला, स..."


In [9]:
Counter(list(df_train[0]))

Counter({'entertainment': 2660, 'sports': 984, 'state': 6028})

In [10]:
df_valid = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/mr/mr-valid.csv', header=None)
df_valid.head()

,0,1
0,sports,"CWG 2018 : संजिता चानूची सुवर्णभरारी, भारताच्य..."
1,state,कर्मचार्‍यांच्या हलगर्जीपणामुळे होणार होते जिव...
2,sports,श्रीलंकेचा भारतावर 7 गडी राखून विजय
3,state,औरंगाबाद महापौरपदाचा तिढा अखेर सुटला
4,state,औरंगाबाद विद्यापीठाचा भोंगळ कारभार; गोदामात दि...


In [11]:
df_test = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/mr/mr-test.csv', header=None)
df_test.head()

,0,1
0,entertainment,काजोल पुन्हा माझ्या आयुष्यात येणार नाही - करण ...
1,entertainment,विराटला चीअर करण्यासाठी अनुष्का सिडनीत
2,entertainment,संतोषचा अड्डा!
3,state,मुरूड समुद्रात बुडालेल्या विद्यार्थ्याचा मृतदे...
4,state,"गोव्यात शिवसेना वेलिंगकरांसोबत,लवकरच युतीची घोषणा"


In [12]:
df_train.shape, df_valid.shape, df_test.shape

((9672, 2), (1210, 2), (1210, 2))

In [13]:
cutoff = int(0.05*len(df_train))

In [14]:
df_train = pd.DataFrame({0: list(df_train[0])[:cutoff], 1: list(df_train[1])[:cutoff]})
df_train.shape

(483, 2)

In [15]:
Counter(list(df_train[0]))

Counter({'entertainment': 136, 'sports': 44, 'state': 303})

In [16]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [17]:
label_cols = [0]

In [18]:
class MarathiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/marathi/tokenizer/marathi_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [19]:
sp = spm.SentencePieceProcessor()
sp.Load(str('./../../models/marathi/tokenizer/marathi_lm.model'))
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [20]:
# 30,000 is the vocab size that we chose in sentencepiece
marathi_vocab = Vocab(itos)

In [21]:
tokenizer = Tokenizer(tok_func=MarathiTokenizer, lang='mr')

In [22]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [23]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=marathi_vocab)

In [24]:
data_lm.show_batch()

idx,text
0,"▁पोलिस ांच्या ▁जाहिराती वर ▁बु म राह ▁संताप ला , ▁सु ना वले ▁खडे बोल ▁x x bo s ▁संतप्त ▁ग्रामस्थ ांनी ▁राष्ट्रवादी ▁आमदार ाच्या ▁दिशे नं ▁भिरकाव ल्या ▁पाण्याने ▁भरलेल्या ▁घागरी ▁x x bo s <unk> <unk> p e c i <unk> al <unk> ▁r e p <unk> or t ▁: ▁लातूर मध्ये ▁का ▁करावा ▁लागतो य ▁विद्यार्थ्यांना ▁एसटी ने ▁लटक ून ▁प्रवास ? ▁x x bo s ▁यश"
1,"▁x x bo s ▁लातूर मध्ये ▁पावसा चा ▁हा हा कार , ▁मन ्या ड ▁नदीच्या ▁ पुरात ▁नागरिक ▁अडकले ▁x x bo s ▁जळगाव , ▁सांगली त ▁55 ▁टक्के ▁मतदान , ▁दोन ▁' दादा ं ' चं ▁वर्चस्व ▁पण ाला ▁x x bo s ▁क्रिकेटपटू ▁मोहम्मद ▁शमी च्या ▁पत्नी ने ▁केला ▁काँग्रेस मध्ये ▁प्रवेश ▁x x bo s ▁ह निम ून ला ▁निघाले ल्या ▁दाम्पत्य ावर ▁पसर णी ▁घाटात"
2,"ंच्या ▁विरोधात ▁अ ब्रु ▁नुकसान ी चा ▁दावा ▁x x bo s ▁हा ॅल ि वूड ▁बा ॅल ि वूड ▁ <unk> ▁14 ▁जानेवारी ▁17 ▁) ▁x x bo s <unk> <unk> ▁g am es ▁2018 : ▁डबल ▁ध मा का , ▁जॉन सन ▁आणि ▁महिला ▁टीम ने ▁पटकावले ▁दोन ▁सुवर्णपदक ▁x x bo s ▁औरंगाबाद मध्ये ▁विद्यार्थ्यांना ▁ओलांड ावा ▁लागतो ▁‘ मृत्यू ’ चा ▁ट्रॅक ▁x x bo s"
3,"▁मोदी ंच्या ▁भूमिके ची ▁आॅफ र ▁x x bo s ▁औरंगाबादेत ▁आंदोलन ाला ▁हिंसक ▁वळण , ▁आं दो लक ांकडून ▁दगड फे क ▁x x bo s ▁सांगली ची ▁' चा ंग ली ' ▁मुलगी ▁स्मृती ची ▁क्रिकेट ▁कहाणी ▁x x bo s ▁पुणे ▁इंटरनॅशनल ▁फिल्म ▁फेस्टिव्हल ला ▁दि मा ख ात ▁सुरुवात ▁x x bo s ▁परभणी त ▁मराठा ▁समाजाचा ▁भव्य ▁मूक ▁मोर्चा ▁x x bo s ▁3 ▁चेहरे"
4,"s ▁ <unk> ▁ba by m ir za m al ik ▁: ▁सान िया - शो ए ब च्या ▁घरी ▁आला ▁नवा ▁पा हु णा ▁x x bo s <unk> rth day <unk> c ial ▁: ▁... म्हणून ▁संजय ▁दत्त ला ▁त्रि शाला चा ▁ताबा ▁मिळाला ▁नव्हता ▁x x bo s ▁आंबे न ळी ▁बस ▁अपघात ापूर्वी ▁दोन वेळा ▁ड्रायव्हर ▁बदलले , ▁दापोली ▁विद्यापीठाचा ▁अहवाल ▁x x bo s ▁कपिल"


In [25]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [26]:
# Loading the pretrained language model on kannada wikipedia
learn.load('../../../models/marathi/lm/ULMFiT/fourth_mr_lm', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (483 items)
x: LMTextList
▁x x bo s ▁... म्हणून ▁सान िया ▁मिर्झा ची ▁ड्यू ▁डे ट ▁आहे ▁खास,▁x x bo s <unk> ▁v ide o ▁: ▁दीपिका - रण वीर च्या ▁लग्न ाचा ▁मे न्यू ▁झाला ▁ली क !,▁x x bo s ▁सचिन च्या ▁रणजी ▁कारकीर्दी चा ▁शेवट ▁गोड , ▁मुंबई चा ▁विजय,▁x x bo s ▁पुरंदरे ंना ▁महाराष्ट्र ▁भूषण ▁पुरस्कार ाविरोधात ▁संभाजी ▁ब्रिगेड ▁रस्त्यावर,▁x x bo s ▁जयपूर ▁पोलिस ांच्या ▁जाहिराती वर ▁बु म राह ▁संताप ला , ▁सु ना वले ▁खडे बोल
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (1210 items)
x: LMTextList
▁x x bo s <unk> ▁c w g ▁2018 ▁: ▁सं जित ा ▁ चा नू ची ▁सुवर्ण भर ारी , ▁भारताच्या ▁खात्यात ▁दुसरे ▁सुवर्ण ▁!,▁x x bo s ▁कर्मचार् ▁यांच्या ▁हल ग र्ज ी पणामुळे ▁होणार ▁होते ▁जिवंत ▁बाळ ावर ▁अंत्यसंस्कार,▁x x bo s ▁श्रीलंकेचा ▁भारतावर ▁7 ▁गडी ▁राखून ▁विजय,▁x x bo s ▁औरंगाबाद ▁महापौर पदाचा ▁ति ढा ▁अखेर ▁सुट ला,▁x x bo s ▁औरंगाबाद ▁विद्यापीठाचा ▁भो ंग ळ ▁कारभार ; ▁गोदाम ात ▁दिले ▁विद्यार्थ्यांनी ▁पेपर
y: LMLabelList
,,,,
Path: .;

Test: LabelList (1210 it

In [27]:
learn.freeze()

In [28]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.366039,6.188520,0.239018,00:00


In [29]:
learn.unfreeze()

In [30]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.115070,6.134318,0.240625,00:00
1,6.037318,5.921357,0.251964,00:00
2,5.908377,5.735169,0.257545,00:00
3,5.769440,5.628993,0.260357,00:00
4,5.657105,5.600702,0.261607,00:00


In [31]:
learn.predict('सीमा भागातील मराठी',n_words=10)

'सीमा भागातील मराठी ▁लोक ▁- ▁x x bo s ▁कृष्णराव ▁सदाशिव ▁कांबळे ▁ऊर्फ'

In [32]:
learn.save_encoder('fine_tuned_enc')

In [33]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=marathi_vocab, label_cols=label_cols, bs=64)

In [34]:
data_clas.show_batch()

text,target
▁x x bo s <unk> ▁l o k ▁ <unk> sa b <unk> ha <unk> ▁e l e c t i o n s ▁20 19 <unk> ▁d <unk> at e : ▁बारामती ▁मतदारसंघ ामध्ये ▁कधी ▁होणार ▁मतदान ?,state
▁x x bo s <unk> ▁v <unk> id e o <unk> ▁j <unk> ac k i e <unk> <unk> h <unk> ro o f ▁: ▁... ▁म्हणून ▁जॅक ी ▁श्रॉफ ▁लखनऊ त ▁झाला ▁ ट्रॅ फ िक ▁हवालदार,entertainment
▁x x bo s <unk> ▁v <unk> id e o ▁: ▁एक ता ▁कपूर च्या ▁' ग ंदी ▁ब ात ▁2 ' च्या ▁ट्रे लर मध्ये ▁से क्स ▁आणि ▁बोल ्ड ▁सी न्स चा ▁भ डी मार,entertainment
"▁x x bo s <unk> ▁b ig <unk> ss ▁12 ▁: ▁ज स लीन नं ▁सगळ्या ं समोर ▁केलं ▁अनुप ▁ज लोट ांना ▁कि स , <unk> ▁v <unk> id e o ▁व्हाय रल",entertainment
▁x x bo s <unk> <unk> p e c i <unk> al <unk> ▁r e p <unk> or t ▁: ▁लातूर मध्ये ▁का ▁करावा ▁लागतो य ▁विद्यार्थ्यांना ▁एसटी ने ▁लटक ून ▁प्रवास ?,state


In [35]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [36]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [37]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (483 items)
x: TextList
▁x x bo s ▁... म्हणून ▁सान िया ▁मिर्झा ची ▁ड्यू ▁डे ट ▁आहे ▁खास,▁x x bo s <unk> ▁v ide o ▁: ▁दीपिका - रण वीर च्या ▁लग्न ाचा ▁मे न्यू ▁झाला ▁ली क !,▁x x bo s ▁सचिन च्या ▁रणजी ▁कारकीर्दी चा ▁शेवट ▁गोड , ▁मुंबई चा ▁विजय,▁x x bo s ▁पुरंदरे ंना ▁महाराष्ट्र ▁भूषण ▁पुरस्कार ाविरोधात ▁संभाजी ▁ब्रिगेड ▁रस्त्यावर,▁x x bo s ▁जयपूर ▁पोलिस ांच्या ▁जाहिराती वर ▁बु म राह ▁संताप ला , ▁सु ना वले ▁खडे बोल
y: CategoryList
entertainment,entertainment,sports,state,sports
Path: .;

Valid: LabelList (1210 items)
x: TextList
▁x x bo s <unk> ▁c w g ▁2018 ▁: ▁सं जित ा ▁ चा नू ची ▁सुवर्ण भर ारी , ▁भारताच्या ▁खात्यात ▁दुसरे ▁सुवर्ण ▁!,▁x x bo s ▁कर्मचार् ▁यांच्या ▁हल ग र्ज ी पणामुळे ▁होणार ▁होते ▁जिवंत ▁बाळ ावर ▁अंत्यसंस्कार,▁x x bo s ▁श्रीलंकेचा ▁भारतावर ▁7 ▁गडी ▁राखून ▁विजय,▁x x bo s ▁औरंगाबाद ▁महापौर पदाचा ▁ति ढा ▁अखेर ▁सुट ला,▁x x bo s ▁औरंगाबाद ▁विद्यापीठाचा ▁भो ंग ळ ▁कारभार ; ▁गोदाम ात ▁दिले ▁विद्यार्थ्यांनी ▁पेपर
y: CategoryList
s

In [38]:
learn.freeze()

In [39]:
learn.loss_func.func

CrossEntropyLoss()

In [40]:
mcc = MatthewsCorreff()

In [41]:
learn.metrics = [mcc, accuracy]

In [42]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.884162,0.914558,0.531952,0.771074,00:01


In [43]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.609857,0.708457,0.421737,0.721488,00:01


In [44]:
learn.save('second-full')

In [45]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.396776,0.656278,0.538668,0.776033,00:01
1,0.353313,0.581845,0.611269,0.809091,00:01
2,0.313213,0.521000,0.677116,0.838843,00:01
3,0.272686,0.487775,0.683356,0.841322,00:01
4,0.248146,0.476638,0.688048,0.842149,00:01


Better model found at epoch 0 with accuracy value: 0.7760330438613892.
Better model found at epoch 1 with accuracy value: 0.8090909123420715.
Better model found at epoch 2 with accuracy value: 0.8388429880142212.
Better model found at epoch 3 with accuracy value: 0.8413223028182983.
Better model found at epoch 4 with accuracy value: 0.8421487808227539.


In [46]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (483 items)
x: TextList
▁x x bo s ▁... म्हणून ▁सान िया ▁मिर्झा ची ▁ड्यू ▁डे ट ▁आहे ▁खास,▁x x bo s <unk> ▁v ide o ▁: ▁दीपिका - रण वीर च्या ▁लग्न ाचा ▁मे न्यू ▁झाला ▁ली क !,▁x x bo s ▁सचिन च्या ▁रणजी ▁कारकीर्दी चा ▁शेवट ▁गोड , ▁मुंबई चा ▁विजय,▁x x bo s ▁पुरंदरे ंना ▁महाराष्ट्र ▁भूषण ▁पुरस्कार ाविरोधात ▁संभाजी ▁ब्रिगेड ▁रस्त्यावर,▁x x bo s ▁जयपूर ▁पोलिस ांच्या ▁जाहिराती वर ▁बु म राह ▁संताप ला , ▁सु ना वले ▁खडे बोल
y: CategoryList
entertainment,entertainment,sports,state,sports
Path: .;

Valid: LabelList (1210 items)
x: TextList
▁x x bo s <unk> ▁c w g ▁2018 ▁: ▁सं जित ा ▁ चा नू ची ▁सुवर्ण भर ारी , ▁भारताच्या ▁खात्यात ▁दुसरे ▁सुवर्ण ▁!,▁x x bo s ▁कर्मचार् ▁यांच्या ▁हल ग र्ज ी पणामुळे ▁होणार ▁होते ▁जिवंत ▁बाळ ावर ▁अंत्यसंस्कार,▁x x bo s ▁श्रीलंकेचा ▁भारतावर ▁7 ▁गडी ▁राखून ▁विजय,▁x x bo s ▁औरंगाबाद ▁महापौर पदाचा ▁ति ढा ▁अखेर ▁सुट ला,▁x x bo s ▁औरंगाबाद ▁विद्यापीठाचा ▁भो ंग ळ ▁कारभार ; ▁गोदाम ात ▁दिले ▁विद्यार्थ्यांनी ▁पेपर
y: CategoryList
s

In [47]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,entertainment,state,sports
0,काजोल पुन्हा माझ्या आयुष्यात येणार नाही - करण ...,entertainment,entertainment,0.874742,0.00781917,0.117439
1,विराटला चीअर करण्यासाठी अनुष्का सिडनीत,entertainment,entertainment,0.708299,0.0280678,0.263633
2,संतोषचा अड्डा!,entertainment,entertainment,0.586619,0.24664,0.16674
3,मुरूड समुद्रात बुडालेल्या विद्यार्थ्याचा मृतदे...,state,state,0.0206947,0.931092,0.0482134
4,"गोव्यात शिवसेना वेलिंगकरांसोबत,लवकरच युतीची घोषणा",state,state,0.0466533,0.871906,0.0814407


In [48]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.8413223140495868

In [49]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.6858955927458129

In [50]:
df_result.to_csv('inltk_headlines_mr_without_aug.csv', index=False)